In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import import_ipynb

pkl_filename = "usefulness-classify-model.pkl"
with open(pkl_filename, 'rb') as file:  
    pkl_linear_svc = pickle.load(file)

X_train_fromCSV = pd.read_csv('x_train.csv')
X_train_fromCSV = X_train_fromCSV.set_index('Unnamed: 0')
X_train_fromCSV = X_train_fromCSV['Body']
# print(X_train_fromCSV.shape, " ", X_train_fromCSV.dtype)
# print("tf_transformer shape: ", tf_transformer.shape)
# transform the data and predict the usefulness with ML model
def predict_usefulness(data):
    count_vect_ = CountVectorizer()
    X_train_counts_ = count_vect_.fit_transform(X_train_fromCSV)
    tf_transformer_ = TfidfTransformer().fit(X_train_counts_)

    #c, t = train_data.fitted()
    p_count = count_vect_.transform([data])
    p_tfidf = tf_transformer_.transform(p_count)
    prediction = pkl_linear_svc.predict(p_tfidf)
    return prediction[0]

importing Jupyter notebook from train_data.ipynb
Hello
 TfidfTransformer()
<function fitted at 0x7fecc3e44ee0>
Average accuracy on test set=0.9677777777777777


<h3>Testing Manually Labeled Data | Number of Emails : 90</h3>

In [2]:
test_90 = pd.read_csv('test_90.csv')
test_90.head()

,Useful,Body,email_subject
0,1.0,"""meeting tmr. join webex""","""cisco hackathon meeting"""
1,1.0,"""You are part of the Group 1 student activatio...","""Important Notification"""
2,0.0,"""The wait is over—your Robinhood tax document ...","""Ready to download"""
3,0.0,"""Hi, We’ve recently launched a new Google Pay ...",NaN
4,0.0,"""I am sending the documents now.""","""Re: Benefits"""


In [3]:
# a method to remove the double quotes from start and end
def remove_quotes(text):
    if text is np.nan: 
        return "no subject"
    if text != None and type(text) != float:
        return text[1:len(text)-1]
    return text

In [4]:
# clean up the data
test_90['Body'] = test_90['Body'].apply(remove_quotes)
test_90['email_subject'] = test_90['email_subject'].apply(remove_quotes)
test_90['Useful'] = test_90['Useful'].fillna(0.0).astype(int)
test_90.head()

,Useful,Body,email_subject
0,1,"meeting tmr. join webex""",cisco hackathon meeting
1,1,You are part of the Group 1 student activation...,Important Notification
2,0,The wait is over—your Robinhood tax document i...,Ready to download
3,0,"Hi, We’ve recently launched a new Google Pay a...",no subject
4,0,I am sending the documents now.,Re: Benefits


In [5]:
body_texts = list(test_90['Body'])

In [6]:
y_true = list(test_90['Useful'])
y_pred = []
count = 0

for idx, body_text in zip(range(len(body_texts)), body_texts):
    y_predict = predict_usefulness(body_text)
    y_pred.append(y_predict)
    if y_predict == y_true[idx]:
        count = count + 1
acc = (count / len(y_true)) * 100, "% Accuracy"
print("Correctly predicted: ", count, " out of ", len(y_true))
print("Accuracy: ", acc)


Correctly predicted:  79  out of  91
Accuracy:  (86.81318681318682, '% Accuracy')


In [7]:
predict_usefulness("This is a friendly reminder that you have an interview tomorrow at 10 AM PST over Zoom meeting. Link can be found in the attached file")

1

In [8]:
predict_usefulness("Hey Min, what do you think of last night game? Did you think they could have a better shot?")

0

In [9]:
predict_usefulness("meeting time!!!!!!!")

1